# Using the Visualizer

The Visualizer is a tool that allows you to visualize the a Resource (like LiquidHandler) including its state to easily see what is going on, for example when executing a protocol on a robot or when developing a new protocol.

When using a backend that does not require access to a physical robot, such as the {class}`~pylabrobot.liquid_handling.backends.chatterbox.ChatterboxBackend`, the Visualizer can be used to simulate a robot's behavior. Of course, you may also use the Visualizer when working with a real robot to see what is happening in the PLR resource and state trackers.

## Setting up a connection with the robot

As described in the [basic liquid handling tutorial](../00_liquid-handling/hamilton-star/basic), we will use the {class}`~pylabrobot.liquid_handling.liquid_handler.LiquidHandler` class to control the robot. This time, however, instead of using the Hamilton {class}`~pylabrobot.liquid_handling.backends.hamilton.STAR.STAR` backend, we are using the software-only {class}`~pylabrobot.liquid_handling.backends.chatterbox.ChatterboxBackend` backend. This means that liquid handling will work exactly the same, but commands are simply printed out to the console instead of being sent to a physical robot. We are still using the same deck.

In [1]:
from pylabrobot.liquid_handling import LiquidHandler
from pylabrobot.visualizer.visualizer import Visualizer

In [2]:
from pylabrobot.resources.opentrons import OTDeck
from pylabrobot.liquid_handling.backends.opentrons_backend import OpentronsBackend

In [3]:
backend = OpentronsBackend(host="localhost", simulation=True)

In [4]:
lh = LiquidHandler(backend=backend, deck=OTDeck())

Calling {func}`~pylabrobot.liquid_handling.liquid_handler.LiquidHandler.setup` will print out "Setting up the robot" and also that two resources were assigned: the deck and the trash. Other than that, the chatter box backend has no setup to do.

In [5]:
await lh.setup()

Next, we will create a {class}`~pylabrobot.visualizer.visualizer.Visualizer` object. The Visualizer expects a Resource, and we will pass the {class}`~pylabrobot.liquid_handling.liquid_handler.LiquidHandler` object to it. This will allow us to visualize the robot's state and actions.

In [6]:
vis = Visualizer(resource=lh)
await vis.setup()

Websocket server started at http://127.0.0.1:2122
File server started at http://127.0.0.1:1338 . Open this URL in your browser.


![The empty simulator](./img/visualizer/empty.png)

## Build the deck layout: Assigning plates and tips

When resources are assigned to the root resource of the Visualizer, in this case `lh`, they will automatically appear in the visualization.

In [7]:
from typing import cast

from pylabrobot.resources.tip_rack import TipRack


In [8]:
from pylabrobot.resources.opentrons import (
  opentrons_96_tiprack_300ul,
  nest_96_wellplate_2ml_deep
)

tip_rack = opentrons_96_tiprack_300ul(name='tip rack')
plt = nest_96_wellplate_2ml_deep(name='plate')

lh.deck.assign_child_at_slot(tip_rack, slot=7)
lh.deck.assign_child_at_slot(plt, slot=4)

![The simulator after the resources have been assigned](./img/visualizer/assignment.png)

### Configuring the state of the deck

As with every PyLabRobot script, you have the option of updating the state of the deck before you actually start your method. This will allow PyLabRobot to keep track of what is going on, enabling features like {func}`~pylabrobot.liquid_handling.liquid_handler.LiquidHandler.return_tips` and catching errors (like missed tips) before a command would be executed on the robot. With the visualizer, this state has the additional effect of updating the visualization.

### Tips

Let's use {func}`~pylabrobot.resources.tip_rack.fill` to place tips at all spots in the tip rack in location `0`.

In [9]:
tip_rack.fill()


You can precisely control the presence of tips using {func}`~pylabrobot.resources.tip_rack.set_tip_state`. This function allows you to set whether there is a tip in each {class}`~pylabrobot.resources.tip_rack.TipSpot`.

In [10]:
#tip_rack.set_tip_state(([True]*8 +[False]*8)*6)

### Liquids

Adding liquid to wells works similarly. You can use {func}`~pylabrobot.resources.plate.set_well_liquids` to set the liquid in each well of a plate. Each liquid is represented by a tuple where the first element corresponds to the type of liquid and the second to the volume in uL. Here, `None` is used to designate an unknown liquid.

In [11]:
plt_1_liquids = [[(None, 300)]]*96
plt.set_well_liquids(plt_1_liquids)

In the visualizer, you can see that the opacity of the well is proportional to how full the well is relative to its maximum volume.

![Simulator after the tips have been placed and the volumes adjusted](./img/visualizer/resources.png)

## Liquid handling

Once the layout is complete, you can run the same commands as described in the [basic liquid handling tutorial](../00_liquid-handling/hamilton-star/basic).

It is important that both tip tracking and volume tracking are enabled globally, so that the visualizer can keep track of the state of the tips and the volumes of the liquids.

In [12]:
from pylabrobot.resources import set_tip_tracking, set_volume_tracking
set_tip_tracking(True), set_volume_tracking(True)

(None, None)

### Picking up tips

Note that since we are using the {class}`~pylabrobot.liquid_handling.backends.chatterbox.ChatterboxBackend`, we just print out "Picking up tips" instead of actually performing an operation. The visualizer will show the tips being picked up.

In [13]:
print("left_pipette:", lh.backend.left_pipette)
print("right_pipette:", lh.backend.right_pipette)
print("left_has_tip:", lh.backend.left_pipette_has_tip)
print("right_has_tip:", lh.backend.right_pipette_has_tip)

left_pipette: {'pipetteId': 'sim_left', 'name': 'p300_single'}
right_pipette: None
left_has_tip: False
right_has_tip: False


In [14]:
await lh.pick_up_tips(tip_rack[0])

TypeError: No run_id given. Please pass run_id as a parameter or use ot_api.set_run_id()

In [22]:
await lh.drop_tips(tip_rack1["A1", "B2", "C3", "D4"])

Dropping tips:
pip#  resource             offset           tip type     max volume (µL)  fitting depth (mm)   tip length (mm)  filter    
  p0: tips_01_tipspot_0_0  0,0,0            HamiltonTip  1065             8                    95.1             Yes       
  p1: tips_01_tipspot_1_1  0,0,0            HamiltonTip  1065             8                    95.1             Yes       
  p2: tips_01_tipspot_2_2  0,0,0            HamiltonTip  1065             8                    95.1             Yes       
  p3: tips_01_tipspot_3_3  0,0,0            HamiltonTip  1065             8                    95.1             Yes       


### Aspirating and dispensing

In [23]:
await lh.pick_up_tips(tip_rack1["A1"])

Picking up tips:
pip#  resource             offset           tip type     max volume (µL)  fitting depth (mm)   tip length (mm)  filter    
  p0: tips_01_tipspot_0_0  0,0,0            HamiltonTip  1065             8                    95.1             Yes       


In [24]:
await lh.aspirate(plate_1["A2"], vols=[200])

Aspirating:
pip#  vol(ul)  resource             offset           flow rate  blowout    lld_z       
  p0: 200.0    plate_01_well_1_0    0,0,0            None       None       None       


In [25]:
await lh.dispense(plate_2["A1"], vols=[200])

Dispensing:
pip#  vol(ul)  resource             offset           flow rate  blowout    lld_z       
  p0: 200.0    plate_02_well_0_0    0,0,0            None       None       None       


In [26]:
await lh.return_tips()

Dropping tips:
pip#  resource             offset           tip type     max volume (µL)  fitting depth (mm)   tip length (mm)  filter    
  p0: tips_01_tipspot_0_0  0,0,0            HamiltonTip  1065             8                    95.1             Yes       


### Aspirating using CoRe 96

The CoRe 96 head supports liquid handling operations for 96 channels at once. Here's how to use:

- {func}`~pylabrobot.liquid_handling.liquid_handler.LiquidHandler.pick_up_tips96` for picking up 96 tips;
- {func}`~pylabrobot.liquid_handling.liquid_handler.LiquidHandler.aspirate96` for aspirating liquid from an entire plate at once;
- {func}`~pylabrobot.liquid_handling.liquid_handler.LiquidHandler.dispense96` for dispensing liquid to an entire plate at once;
- {func}`~pylabrobot.liquid_handling.liquid_handler.LiquidHandler.drop_tips96` for dropping tips to the tip rack.


In [27]:
await lh.pick_up_tips96(tip_rack1)

Picking up tips from tips_01.


In [28]:
await lh.aspirate96(plate_1, volume=100)

Aspirating 100.0 from Plate(name=plate_01, size_x=127.76, size_y=85.48, size_z=14.2, location=Coordinate(000.000, 000.000, -03.030)).


In [29]:
await lh.dispense96(plate_3, volume=100)

Dispensing 100.0 to Plate(name=plate_03, size_x=127.76, size_y=85.48, size_z=14.2, location=Coordinate(000.000, 000.000, -03.030)).


In [30]:
await lh.drop_tips96(tip_rack1)

Dropping tips to tips_01.


![The simulator after the liquid handling operations completed](./img/visualizer/after_lh.png)

## Shutting down

When you're done, you can stop the visualizer by calling {func}`~pylabrobot.visualizer.visualizer.Visualizer.stop`. This will stop the visualization.

In [13]:
await vis.stop()